In [1]:
import pandas as pd

In [41]:
from datetime import datetime

In [112]:
import numpy as np

## I guess we're trying to figure out who is stealing fuel
- if fuel is clandestinely leaving a fuel tank without a transaction record then...
    - the truck/unit will be traveling less miles than is expected with purchased fuel
    - so, presumably, I should be able to compare a unit's "apparent" calculated mpg with it's nominal mpg
    - units with unrealistically low mpg's would be the 'donors' of untracked fuel
        - perhaps there are even trucks with unrealistically high mpgs who are "receivers" of untracked fuel
    - So, Do I have records of fuel consumption and fuel miles traveled?

## --read and rename csvs and columns
All data is from Jan. 1, 2017 - March, 23, 2018

In [2]:
#read in the csvs
events_fuel_df = pd.read_csv('data/Extranet2.ExactFuelEvents.csv')
    # fuel events 8.5M rows
        # sensors send readings every 15 minutes and at key on and key off

In [3]:
events_fuel_df.head()

,Id,ESS_Id,EventTimeStamp,EquipmentID,MCTNumber,EquipmentType,DriverID,Latitude,Longitude,LocationTimeStamp,Speed,Heading,Odometer,IgnitionStatus,EFReportReason
0,12649083,13640241,2017-01-01 00:00:32.387,1992,105369677,tractor,BATK,30.34642,-81.70724,2017-01-01 00:00:30.000,0,0.0,39895.5,2,1
1,12649084,13640242,2017-01-01 00:01:24.867,1712,105437938,tractor,TAYANT,36.93953,-84.09541,2017-01-01 00:01:27.000,0,0.0,360378.8,1,2
2,12649092,13640251,2017-01-01 00:04:15.053,1713,105438293,tractor,RUSR,35.58664,-84.52786,2017-01-01 00:04:17.000,67,219.5,377577.4,1,2
3,12649102,13640261,2017-01-01 00:04:10.260,1585,105301976,tractor,MCAD,35.15430,-90.14263,2017-01-01 00:04:10.000,0,0.0,513931.8,1,2
4,12649103,13640272,2017-01-01 00:10:13.187,1564,105351785,tractor,ULAL,39.90268,-84.19254,2017-01-01 00:10:11.000,3,91.0,498691.0,1,2


In [4]:
#events_fuel_df.info()

In [5]:
# fuel levels 7.9 M rows
levels_fueltank_df = pd.read_csv('data/Extranet2.ExactFuelTankLevels.csv')
    # Some trucks have sensors in one tank, some in both. In either case
    # two readings are always transmitted, and if TankLevelGallons = 0
    # no record is written.
        # if record in lefels_fueltank_df (ExactFuelTankLevels) does not 
        # have a corresponding record in (TankLevelGallons) this is likely
        # due to a connection error either in communicating with the truck
        # or writing to the database

In [6]:
levels_fueltank_df.head()

,Id,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
0,10391480,1,82.8,165.60,12667508
1,10391482,1,72.4,144.80,12667510
2,10391505,1,57.1,114.20,12667533
3,10391507,1,80.9,161.80,12667535
4,10391514,1,56.9,142.25,12667542


In [94]:
def trim_categories(df, cols_list):
    for col in cols_list:
        df[col].cat.rename_categories(df[col].cat.categories.str.strip(), inplace=True)

In [95]:
# select UNUNIT, UNYEAR, UNMAKE, UNSER from IBGEFILE.UNITS 
#where (UNTERM = 0 OR UNTERM >= 2017001) 
#and UNUNIT NO LIKE 'T%' and trim (UNUNIT) NOT LIKE '99%'
units_df = pd.read_csv('data/IBGEFILE.UNITS.csv', 
                       dtype = {'UNMAKE': 'category'})
#Can drop non-int units
units_df.columns = ['unit_number', 'model_year', 'make'] 

# Trim white space from category strings 

# Remove invalid entries
units_df.unit_number = pd.to_numeric(units_df.unit_number, errors = 'coerce')
units_df = units_df[units_df.unit_number != 1].dropna().set_index('unit_number')
#units_df = units_df[units_df.unit_number.isin(['ZEBE', 'CCTRK', 'BGXL', '1']) == False]
#set index

trim_categories(units_df, ['make'])
# Fix typos
units_df.make[units_df.make.isin(['KENOWRTH', 'KEWORTH'])] = 'KENWORTH'

In [96]:
units_df.head()

,model_year,make
unit_number,,
535.0,1996,PETERBILT
534.0,1996,PETERBILT
533.0,1996,PETERBILT
532.0,1996,PETERBILT
531.0,1996,PETERBILT


In [9]:
optimization_fuel_df = pd.read_csv('data/BGEIDSC.EF2EFFP.csv')
#DB2 table from Fuel Optimization vendor


In [10]:
#column names to lower for easier typing
optimization_fuel_df.columns = [s.lower() for s in optimization_fuel_df.columns]
optimization_fuel_df.columns

Index(['truck', 'active', 'sendfuel', 'sendroute', 'tankcap', 'avgmpg',
       'sattype'],
      dtype='object')

In [11]:
optimization_fuel_df.head()

,truck,active,sendfuel,sendroute,tankcap,avgmpg,sattype
0,1,Y,Y,Y,200,6.0,
1,243,Y,Y,Y,200,6.0,
2,244,Y,Y,Y,200,6.0,
3,245,Y,Y,Y,200,6.0,
4,246,Y,Y,Y,200,6.0,


In [26]:
transaction_df = pd.read_csv('data/BGETCHDATA.PTCHTRANH.csv')
#Actual fuel transactions will have an item code of:
act_fuel_transactions = ['ULSD', 'FUEL', 'CDSL', 'DSL1', 'BDSL']
# and could be in any of the item slots 1-6

In [27]:
transaction_df.columns = ['tr_id', 'tr_date', 'tr_time', 'tr_station_id', 'tr_station_name',
                         'tr_station_cty', 'tr_station_st', 'tr_drvr_code', 'tr_unit_id', 
                         'tr_item_cd1', 'tr_item_qty1', 'tr_item_cd2', 'tr_item_qty2',
                          'tr_item_cd3', 'tr_item_qty3', 'tr_item_cd4', 'tr_item_qty4',
                          'tr_item_cd5', 'tr_item_qty5', 'tr_item_cd6', 'tr_item_qty6',
                         ]

In [14]:
transaction_df.head()

,tr_id,tr_date,tr_time,tr_station_id,tr_station_name,tr_station_cty,tr_station_st,tr_drvr_code,tr_unit_id,tr_item_cd1,...,tr_item_cd2,tr_item_qty2,tr_item_cd3,tr_item_qty3,tr_item_cd4,tr_item_qty4,tr_item_cd5,tr_item_qty5,tr_item_cd6,tr_item_qty6
0,42574765,20170101,20,520277,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,...,,0.00,,0.0,,0.0,,0.0,,0
1,42575331,20170101,48,516202,PILOT NEW BRAUNFELS 330,NEW BRAUNFELS,TX,MANRI,1584,ULSD,...,DEFD,7.71,CADV,1.0,,0.0,,0.0,,0
2,42578678,20170101,134,516277,PILOT 421,DALTON,GA,RUSR,1713,ULSD,...,DEFD,4.12,,0.0,,0.0,,0.0,,0
3,42578722,20170101,137,231100,FJ-BRUNSWICK 627,BRUNSWICK,GA,FOSJA,1711,ULSD,...,,0.00,,0.0,,0.0,,0.0,,0
4,42579695,20170101,302,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,HICWIL,1837,ULSD,...,,0.00,,0.0,,0.0,,0.0,,0


In [15]:
data_performance = pd.read_csv('data/Extranet2.QCPerformanceExtracts.csv',
                              header = None)

FileNotFoundError: File b'data/Extranet2.QCPerformanceExtracts.csv' does not exist

In [ ]:
data_performance

In [ ]:
##########---------converting cols to datetime-------
def cols_to_timestamp(df, col_list):
    for col in col_list:
        df[col] = pd.to_datetime(df[col], format = '%Y-%m-%d %H:%M:%S')
    return df
#ts_cols = ['EventTimeStamp', 'LocationTimeStamp']
#events_fuel_df =  cols_to_timestamp(events_fuel_df, ts_cols)  

## Building master_df from our data

In [16]:
#transaction_df.tr_id identify each fuel purchase from the fuel vendor software
    #data/BGETCHDATA.PTCHTRANH.csv
transaction_df.head(3)

,tr_id,tr_date,tr_time,tr_station_id,tr_station_name,tr_station_cty,tr_station_st,tr_drvr_code,tr_unit_id,tr_item_cd1,...,tr_item_cd2,tr_item_qty2,tr_item_cd3,tr_item_qty3,tr_item_cd4,tr_item_qty4,tr_item_cd5,tr_item_qty5,tr_item_cd6,tr_item_qty6
0,42574765,20170101,20,520277,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,...,,0.00,,0.0,,0.0,,0.0,,0
1,42575331,20170101,48,516202,PILOT NEW BRAUNFELS 330,NEW BRAUNFELS,TX,MANRI,1584,ULSD,...,DEFD,7.71,CADV,1.0,,0.0,,0.0,,0
2,42578678,20170101,134,516277,PILOT 421,DALTON,GA,RUSR,1713,ULSD,...,DEFD,4.12,,0.0,,0.0,,0.0,,0


In [28]:
transaction_df.info() 
#showed that tr_time was an int

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147861 entries, 0 to 147860
Data columns (total 21 columns):
tr_id              147861 non-null int64
tr_date            147861 non-null int64
tr_time            147861 non-null int64
tr_station_id      147861 non-null int64
tr_station_name    147861 non-null object
tr_station_cty     147861 non-null object
tr_station_st      147861 non-null object
tr_drvr_code       147861 non-null object
tr_unit_id         147861 non-null object
tr_item_cd1        147861 non-null object
tr_item_qty1       147861 non-null float64
tr_item_cd2        147861 non-null object
tr_item_qty2       147861 non-null float64
tr_item_cd3        147861 non-null object
tr_item_qty3       147861 non-null float64
tr_item_cd4        147861 non-null object
tr_item_qty4       147861 non-null float64
tr_item_cd5        147861 non-null object
tr_item_qty5       147861 non-null float64
tr_item_cd6        147861 non-null object
tr_item_qty6       147861 non-null int64
dtypes:

In [29]:
#confirm no junk data in this col. 
None in transaction_df.tr_time
#transaction_df.tr_time.value_counts()

False

In [30]:
#Combine tr_date and tr_time into a single date time column
    #1. zero pad tr_time on left for interpretation as military time
        #cast tr_time as string
transaction_df.tr_time = transaction_df.tr_time.astype(str)

In [31]:
#ensure none of my strings are over 4 digits in length
for s in transaction_df.tr_time:
    assert len(s) < 5

In [32]:
#check for blanks
'' in transaction_df.tr_time

False

In [35]:
#add zeros to the left until each cell is len 4
zero_padded_time = []
for s in transaction_df.tr_time:
    while len(s) < 4:
        s = '0' + s
    zero_padded_time.append(s)

In [36]:
#spot check
for s in zero_padded_time:
    assert len(s) == 4

In [37]:
transaction_df.tr_time = zero_padded_time
transaction_df.tr_time

0         0020
1         0048
2         0134
3         0137
4         0302
5         0307
6         0315
7         0430
8         0430
9         0437
10        0446
11        0453
12        0516
13        0520
14        0530
15        0530
16        0531
17        0532
18        0543
19        0559
20        0621
21        0625
22        0648
23        0711
24        0713
25        0719
26        0723
27        0731
28        0742
29        0743
          ... 
147831    0633
147832    0637
147833    0638
147834    0638
147835    0639
147836    0641
147837    0653
147838    0657
147839    0659
147840    0701
147841    0703
147842    0708
147843    0708
147844    0713
147845    0714
147846    0715
147847    0719
147848    0722
147849    0723
147850    0725
147851    0729
147852    0734
147853    0734
147854    0736
147855    0744
147856    0745
147857    0748
147858    0750
147859    0752
147860    0753
Name: tr_time, Length: 147861, dtype: object

In [38]:
transaction_df.head(2)

,tr_id,tr_date,tr_time,tr_station_id,tr_station_name,tr_station_cty,tr_station_st,tr_drvr_code,tr_unit_id,tr_item_cd1,...,tr_item_cd2,tr_item_qty2,tr_item_cd3,tr_item_qty3,tr_item_cd4,tr_item_qty4,tr_item_cd5,tr_item_qty5,tr_item_cd6,tr_item_qty6
0,42574765,20170101,0020,520277,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,...,,0.00,,0.0,,0.0,,0.0,,0
1,42575331,20170101,0048,516202,PILOT NEW BRAUNFELS 330,NEW BRAUNFELS,TX,MANRI,1584,ULSD,...,DEFD,7.71,CADV,1.0,,0.0,,0.0,,0


In [39]:
transaction_df.tr_date

0         20170101
1         20170101
2         20170101
3         20170101
4         20170101
5         20170101
6         20170101
7         20170101
8         20170101
9         20170101
10        20170101
11        20170101
12        20170101
13        20170101
14        20170101
15        20170101
16        20170101
17        20170101
18        20170101
19        20170101
20        20170101
21        20170101
22        20170101
23        20170101
24        20170101
25        20170101
26        20170101
27        20170101
28        20170101
29        20170101
            ...   
147831    20180323
147832    20180323
147833    20180323
147834    20180323
147835    20180323
147836    20180323
147837    20180323
147838    20180323
147839    20180323
147840    20180323
147841    20180323
147842    20180323
147843    20180323
147844    20180323
147845    20180323
147846    20180323
147847    20180323
147848    20180323
147849    20180323
147850    20180323
147851    20180323
147852    20

In [60]:
#append tr_date (as string) and tr_time together, and then convert them to dateime
#transaction_df.tr_datetime 
tst = [datetime.strptime(str(d) + 
                         t, '%Y%m%d%H%M') 
       for d, t in zip(transaction_df.tr_date,
                       transaction_df.tr_time)]
#love list comprehensions

In [75]:
tst[0:5]#spot check

[datetime.datetime(2017, 1, 1, 0, 20),
 datetime.datetime(2017, 1, 1, 0, 48),
 datetime.datetime(2017, 1, 1, 1, 34),
 datetime.datetime(2017, 1, 1, 1, 37),
 datetime.datetime(2017, 1, 1, 3, 2)]

In [81]:
transaction_df['tr_datetime'] = tst#assign

In [82]:
transaction_df.columns

Index(['tr_id', 'tr_station_id', 'tr_station_name', 'tr_station_cty',
       'tr_station_st', 'tr_drvr_code', 'tr_unit_id', 'tr_item_cd1',
       'tr_item_qty1', 'tr_item_cd2', 'tr_item_qty2', 'tr_item_cd3',
       'tr_item_qty3', 'tr_item_cd4', 'tr_item_qty4', 'tr_item_cd5',
       'tr_item_qty5', 'tr_item_cd6', 'tr_item_qty6', 'tr_datetime'],
      dtype='object')

In [83]:
#drop superfluous columns
transaction_df.drop(['tr_date','tr_time'], axis = 'columns', inplace=True)
transaction_df.head()

,tr_id,tr_station_id,tr_station_name,tr_station_cty,tr_station_st,tr_drvr_code,tr_unit_id,tr_item_cd1,tr_item_qty1,tr_item_cd2,tr_item_qty2,tr_item_cd3,tr_item_qty3,tr_item_cd4,tr_item_qty4,tr_item_cd5,tr_item_qty5,tr_item_cd6,tr_item_qty6,tr_datetime
0,42574765,520277,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,1.00,,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 00:20:00
1,42575331,516202,PILOT NEW BRAUNFELS 330,NEW BRAUNFELS,TX,MANRI,1584,ULSD,137.40,DEFD,7.71,CADV,1.0,,0.0,,0.0,,0,2017-01-01 00:48:00
2,42578678,516277,PILOT 421,DALTON,GA,RUSR,1713,ULSD,137.93,DEFD,4.12,,0.0,,0.0,,0.0,,0,2017-01-01 01:34:00
3,42578722,231100,FJ-BRUNSWICK 627,BRUNSWICK,GA,FOSJA,1711,ULSD,45.00,,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 01:37:00
4,42579695,523050,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,HICWIL,1837,ULSD,107.36,,0.00,,0.0,,0.0,,0.0,,0,2017-01-01 03:02:00


In [108]:
units_df.head(3)

,unit_number,model_year,make
0,535.0,1996,PETERBILT
1,534.0,1996,PETERBILT
2,533.0,1996,PETERBILT


In [109]:
units_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 3 columns):
unit_number    704 non-null float64
model_year     704 non-null object
make           704 non-null category
dtypes: category(1), float64(1), object(1)
memory usage: 12.1+ KB


In [104]:
units_df.reset_index(level = 0, inplace = True)
units_df.head()

,unit_number,model_year,make
0,535.0,1996,PETERBILT
1,534.0,1996,PETERBILT
2,533.0,1996,PETERBILT
3,532.0,1996,PETERBILT
4,531.0,1996,PETERBILT


In [113]:
units_df.unit_number.astype(np.int)
units_df.head()

,unit_number,model_year,make
0,535.0,1996,PETERBILT
1,534.0,1996,PETERBILT
2,533.0,1996,PETERBILT
3,532.0,1996,PETERBILT
4,531.0,1996,PETERBILT


In [117]:
units_df.unit_number.value_counts().head()

2174.0    1
2146.0    1
2136.0    1
2088.0    1
2072.0    1
Name: unit_number, dtype: int64

In [118]:
transaction_df.tr_unit_id.astype(np.int, coerce = True)

ValueError: invalid literal for int() with base 10: 'OFFICE'

In [106]:
#this isn't working yet
#master_df = pd.merge(transaction_df, units_df, how = 'outer',
                    left_on = 'tr_unit_id', 
                     right_on = 'unit_number')
                    #right_index = True)

In [107]:
master_df.head()

,tr_id,tr_station_id,tr_station_name,tr_station_cty,tr_station_st,tr_drvr_code,tr_unit_id,tr_item_cd1,tr_item_qty1,tr_item_cd2,...,tr_item_cd4,tr_item_qty4,tr_item_cd5,tr_item_qty5,tr_item_cd6,tr_item_qty6,tr_datetime,unit_number,model_year,make
0,42574765.0,520277.0,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,1.00,,...,,0.0,,0.0,,0.0,2017-01-01 00:20:00,NaN,NaN,NaN
1,42614106.0,516130.0,PILOT NITRO 243,NITRO,WV,ULAL,1564,ULSD,76.87,,...,,0.0,,0.0,,0.0,2017-01-01 17:12:00,NaN,NaN,NaN
2,42672413.0,524100.0,PFJ SOUTHEAST 7976,TROUTMAN,NC,ULAL,1564,DEFD,12.37,,...,,0.0,,0.0,,0.0,2017-01-02 15:29:00,NaN,NaN,NaN
3,42873010.0,516012.0,PILOT LAVERGNE 052,LAVERGNE,TN,ULAL,1564,ULSD,50.58,CADV,...,,0.0,,0.0,,0.0,2017-01-04 11:52:00,NaN,NaN,NaN
4,42882523.0,523050.0,BIG G EXPRESS TERMINAL,GLADEVILLE,TN,ULAL,1564,ULSD,141.73,,...,,0.0,,0.0,,0.0,2017-01-04 13:00:00,NaN,NaN,NaN
